In [1]:
import yt_dlp
import json
import os
from datetime import datetime, timedelta
from urllib.error import HTTPError


This file will download all videos and live streams in the past 24 hours (skipping over any previously downloaded by using the json files within each folder to track the ID's of previously downloaded videos)

In [2]:
# All channel names (weird name for buccaneers because of they way it is set up)
channel_names = ["49ers", "AtlantaFalcons", "azcardinals", "BaltimoreRavens", "Bengals", "broncos", "browns", "channel/UC0Wwu7r1ybaaR09ANhudTzA", 
                "buffalobills", "CarolinaPanthers", "chargers", "ChicagoBears", "colts", "commandersnfl", "DallasCowboys", 
                "detroitlionsnfl", "eagles", "HoustonTexans", "jaguars", "KansasCityChiefs", "LARams", "MiamiDolphins", 
                "NewOrleansSaints","NewYorkGiants", "nyjets", "packers", "patriots", "raiders", "Seahawks", "steelers", 
                "Titans", "vikings"]

# Path to the directory containing ffmpeg and ffprobe executables (if not in PATH)
ffmpeg_location = 'c:/users/12505/anaconda3/lib/site-packages/ffmpeg/bin'
#Limiting amount of videos to fetch. No need to fetch all the videos
max_videos_to_fetch = 10

In [3]:
# load the already downloaded videos that have their ids stored in the json file
def load_downloaded_videos(record_file):
    if os.path.exists(record_file):
        with open(record_file, 'r') as f:
            return json.load(f)
    return []

In [4]:
# will save the ids of any newly downloaded videos within the json file
def save_downloaded_videos(record_file, downloaded_videos):
    with open(record_file, 'w') as f:
        json.dump(downloaded_videos, f)

In [5]:
# convert the video to audio format using some options ydl offers
def download_audio(url, output_path):
    ydl_opts = {
        'outtmpl': output_path,
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'ffmpeg_location': ffmpeg_location,  # Specify the path to ffmpeg and ffprobe if necessary
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
    except Exception as e:
        print(f"Error downloading {url}: {e}")

In [6]:
# will get the latest videos uploaded to a channel and only select those within a 24 hour period. 
# if the video has already been downloaded the function will skip it.
def get_latest_videos(channel_url, downloaded_videos):
    ydl_opts = {
        'quiet': True,
        'extract_flat': 'in_playlist',
        'playlistend': max_videos_to_fetch,
        'skip_download': True,
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        result = ydl.extract_info(channel_url, download=False)
    one_day_ago = datetime.now() - timedelta(days=1)
    print(one_day_ago)
    new_videos = []
    print(f"Checking {len(result['entries'])} videos from the channel...{channel_url}")

    for entry in result['entries']:
        video_id = entry['id']
        video_url = f"https://www.youtube.com/watch?v={video_id}"
        try:
            # Get detailed info for each video to check upload date
            with yt_dlp.YoutubeDL({'quiet': True}) as ydl:
                video_info = ydl.extract_info(video_url, download=False)
            upload_date = datetime.strptime(video_info['upload_date'], '%Y%m%d')

            if upload_date > one_day_ago and video_id not in downloaded_videos:
                print(f"Adding video {video_url}, uploaded on {upload_date}")
                new_videos.append(video_url)
            else:
                print(f"Skipping video {video_url}, uploaded on {upload_date}")
                break
        except Exception as e:
            print(f"Error processing video {video_url}: {e}")
    
    return new_videos

In [7]:
# runs and downloads videos appropriately
def main():
    for channel_name in channel_names:
        output_dir = f'videos/{channel_name}/%(title)s.%(ext)s'

        # Path to save the record of downloaded video IDs
        record_file = f'videos/{channel_name}/downloaded_videos.json'

        # Directory to save downloaded audio files
        output_dir = f'videos/{channel_name}/%(title)s.%(ext)s'

        # Channel URL
        if(channel_name == "channel/UC0Wwu7r1ybaaR09ANhudTzA" or channel_name == "detroitlionsnfl"):
            print("here")
            channel_url = f'https://www.youtube.com/{channel_name}/videos'
        else:
            print("yo")
            channel_url = f'https://www.youtube.com/c/{channel_name}/videos'
                    
        # To Get Live Streams
        if(channel_name == "channel/UC0Wwu7r1ybaaR09ANhudTzA" or channel_name == "detroitlionsnfl"):
            print("here1")
            live_url = f'https://www.youtube.com/{channel_name}/streams'
        else:
            print("yo1")
            live_url = f'https://www.youtube.com/c/{channel_name}/streams'
            
        downloaded_videos = load_downloaded_videos(record_file)
        latest_videos = get_latest_videos(channel_url, downloaded_videos)
        latest_live = get_latest_videos(live_url, downloaded_videos)

        try:
            for video_url in latest_videos: #get latest videos
                print(f"Downloading audio for {video_url}")
                download_audio(video_url, output_dir)
                video_id = video_url.split('=')[1]
                downloaded_videos.append(video_id)
                
            for video_url in latest_live: #get latest live streams
                print(f"Downloading audio for {video_url}")
                download_audio(video_url, output_dir)
                video_id = video_url.split('=')[1]
                downloaded_videos.append(video_id)         
            
            save_downloaded_videos(record_file, downloaded_videos)
        except Exception as e: 
            print(f"Error processing video {video_url}: {e}")
            continue
        

if __name__ == "__main__":
    main()
    print("Done!")


yo
yo1
2024-05-28 15:09:03.067596
Checking 10 videos from the channel...https://www.youtube.com/c/49ers/videos


KeyboardInterrupt: 

In [ ]:
# def download_audio(url, output_path):
#     ydl_opts = {
#         'outtmpl': output_path,
#         'format': 'bestaudio/best',
#         'postprocessors': [{
#             'key': 'FFmpegExtractAudio',
#             'preferredcodec': 'mp3',
#             'preferredquality': '192',
#         }],
#         'ffmpeg_location' : ffmpeg_location,
#     }
#     try:
#         with yt_dlp.YoutubeDL(ydl_opts) as ydl:
#             ydl.download([url])
#     except Exception as e:
#         print(f"Error downloading {url}: {e}")

In [ ]:
# ffmpeg_location = 'c:/users/12505/anaconda3/lib/site-packages/ffmpeg/bin'
# # List of YouTube URLs to download
# urls = [
#     'https://www.youtube.com/watch?v=r7jgEE6t1u4&ab_channel=NewOrleansSaints',
# ]

# # Directory to save downloaded videos
# output_dir = '/videos/%(title)s.%(ext)s'

# for url in urls:
#     download_audio(url, output_dir)